In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes transformers

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-pt2vpms7/unsloth_031e87e953ce49f59f036018986cc34f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-pt2vpms7/unsloth_031e87e953ce49f59f036018986cc34f
  Resolved https://github.com/unslothai/unsloth.git to commit 28e9de27d7f16b25ab72eef2bcf29c16f05a905a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

llama_model, llama_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Antonio27/llama3-8b-4-bit-for-sugar",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

gemma_model, gemma_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-9b-it-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/132k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [3]:
# Init the test variables

question_1="What makes a rainbow?"
question_2="What are stars made of?"
question_3="How do airplanes fly?"
question_4="Why do we need to exercise?"
question_5="How do computers work?"

alpaca_prompt = """Below is an instruction that describes a task, along with an input that provides additional context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [25]:
FastLanguageModel.for_inference(llama_model)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.add_eos_token = True

inputs = llama_tokenizer(
    [
        alpaca_prompt.format(
            f'''
            Your task is to answer children's questions using simple language.
            Explain any difficult words in a way a 3-year-old can understand.
            Keep responses under 60 words.
            \n\nQuestion: {question_1}
            ''',  # instruction
            "",  # input
            "",  # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

outputs = llama_model.generate(**inputs, max_new_tokens=256, temperature=0.6)
decoded_outputs = llama_tokenizer.batch_decode(outputs)

response_text = decoded_outputs[0]
# response_start = response_text.find("### Response:") + len("### Response:")
# initial_response = response_text[response_start:].strip()

match = re.search(r"### Response:(.*?)(?=\n###|$)", response_text, re.DOTALL)
if match:
    initial_response = match.group(1).strip()
else:
    initial_response = ""

print("Initial Response:", initial_response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Initial Response: A rainbow is a beautiful sight in the sky. It is made up of many colors. The colors are created when light is reflected and refracted by water droplets in the air. When the sun is shining and there are clouds in the sky, the water droplets act like tiny prisms, breaking up the sunlight into a spectrum of colors. The different colors appear in a specific order: red, orange, yellow, green, blue, indigo, and violet. The rainbow is a beautiful sight that can be seen from miles away. It is a reminder of the beauty and diversity of nature.<|end_of_text|>


In [26]:
FastLanguageModel.for_inference(gemma_model)
gemma_tokenizer.pad_token = gemma_tokenizer.eos_token
gemma_tokenizer.add_eos_token = True

inputs = gemma_tokenizer(
    [
        alpaca_prompt.format(
            f'''
            Modify the given content for a 5-year-old.
            Use simple words and phrases.
            Remove any repetitive information.
            Keep responses under 50 words.
            \n\nGiven Content: {initial_response}
            ''',  # instruction
            "",  # input
            "",  # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

outputs = gemma_model.generate(**inputs, max_new_tokens=256, temperature=0.6)
decoded_outputs = gemma_tokenizer.batch_decode(outputs)

response_text = decoded_outputs[0]

match = re.search(r"### Response:(.*?)(?=\n###|$)", response_text, re.DOTALL)
if match:
    adjusted_response = match.group(1).strip()
else:
    adjusted_response = ""

print("Adjusted Response:", adjusted_response)

Adjusted Response: <eos> 

Rainbows are colorful arcs in the sky! They appear when the sun shines after rain. Water drops in the air act like tiny mirrors, making the sunlight look like all the colors of the rainbow. 


<end_of_turn><eos>


In [28]:
FastLanguageModel.for_inference(llama_model)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.add_eos_token = True

inputs = llama_tokenizer(
    [
        alpaca_prompt.format(
            f'''
            Your task is to answer children's questions using simple language.
            Explain any difficult words in a way a 3-year-old can understand.
            Keep responses under 60 words.
            \n\nQuestion: {question_2}
            ''',  # instruction
            "",  # input
            "",  # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

outputs = llama_model.generate(**inputs, max_new_tokens=256, temperature=0.6)
decoded_outputs = llama_tokenizer.batch_decode(outputs)

response_text = decoded_outputs[0]
# response_start = response_text.find("### Response:") + len("### Response:")
# initial_response = response_text[response_start:].strip()

match = re.search(r"### Response:(.*?)(?=\n###|$)", response_text, re.DOTALL)
if match:
    initial_response = match.group(1).strip()
else:
    initial_response = ""

print("Initial Response:", initial_response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Initial Response: Stars are made of hydrogen and helium.
            They are the most common elements in the universe.
            When a star is born, it is made of these elements.
            The star's gravity pulls the elements together,
            and they become hotter and hotter.
            The star's surface becomes hotter and hotter,
            until it is so hot that it can't shine anymore.
            The star has become a black hole.
            
<|end_of_text|>


In [29]:
FastLanguageModel.for_inference(gemma_model)
gemma_tokenizer.pad_token = gemma_tokenizer.eos_token
gemma_tokenizer.add_eos_token = True

inputs = gemma_tokenizer(
    [
        alpaca_prompt.format(
            f'''
            Modify the given content for a 5-year-old.
            Use simple words and phrases.
            Remove any repetitive information.
            Keep responses under 50 words.
            \n\nGiven Content: {initial_response}
            ''',  # instruction
            "",  # input
            "",  # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

outputs = gemma_model.generate(**inputs, max_new_tokens=256, temperature=0.6)
decoded_outputs = gemma_tokenizer.batch_decode(outputs)

response_text = decoded_outputs[0]

match = re.search(r"### Response:(.*?)(?=\n###|$)", response_text, re.DOTALL)
if match:
    adjusted_response = match.group(1).strip()
else:
    adjusted_response = ""

print("Adjusted Response:", adjusted_response)

Adjusted Response: <eos> 

Stars are made of special stuff called hydrogen and helium. They are very common in space! When a star is born, it gets really, really hot and shines brightly. 


<end_of_turn><eos>


In [30]:
FastLanguageModel.for_inference(llama_model)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.add_eos_token = True

inputs = llama_tokenizer(
    [
        alpaca_prompt.format(
            f'''
            Your task is to answer children's questions using simple language.
            Explain any difficult words in a way a 3-year-old can understand.
            Keep responses under 60 words.
            \n\nQuestion: {question_3}
            ''',  # instruction
            "",  # input
            "",  # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

outputs = llama_model.generate(**inputs, max_new_tokens=256, temperature=0.6)
decoded_outputs = llama_tokenizer.batch_decode(outputs)

response_text = decoded_outputs[0]
# response_start = response_text.find("### Response:") + len("### Response:")
# initial_response = response_text[response_start:].strip()

match = re.search(r"### Response:(.*?)(?=\n###|$)", response_text, re.DOTALL)
if match:
    initial_response = match.group(1).strip()
else:
    initial_response = ""

print("Initial Response:", initial_response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Initial Response: Planes fly by moving through the air. The wings are designed to create lift, which allows the plane to fly. The engines provide thrust, which helps the plane move through the air. The plane's shape and design also play a role in its ability to fly. The plane's wings are curved to create lift, and the tail helps to stabilize the plane. The plane's weight also plays a role, as a heavier plane will need more lift to fly. The plane's size and shape also affect its ability to fly. A smaller plane will need less lift than a larger plane. The plane's engine and fuel also play a role, as the plane needs enough fuel to fly. The plane's altitude also affects its ability to fly. The plane needs to be at a high enough altitude to fly, but not too high, as the air becomes too thin. The plane's speed also plays a role, as the plane needs to be moving fast enough to create lift. The plane's control surfaces also play a role, as they help the pilot to control the plane's direction an

In [31]:
FastLanguageModel.for_inference(gemma_model)
gemma_tokenizer.pad_token = gemma_tokenizer.eos_token
gemma_tokenizer.add_eos_token = True

inputs = gemma_tokenizer(
    [
        alpaca_prompt.format(
            f'''
            Modify the given content for a 5-year-old.
            Use simple words and phrases.
            Remove any repetitive information.
            Keep responses under 50 words.
            \n\nGiven Content: {initial_response}
            ''',  # instruction
            "",  # input
            "",  # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

outputs = gemma_model.generate(**inputs, max_new_tokens=256, temperature=0.6)
decoded_outputs = gemma_tokenizer.batch_decode(outputs)

response_text = decoded_outputs[0]

match = re.search(r"### Response:(.*?)(?=\n###|$)", response_text, re.DOTALL)
if match:
    adjusted_response = match.group(1).strip()
else:
    adjusted_response = ""

print("Adjusted Response:", adjusted_response)

Adjusted Response: <eos> to fly. 


Planes have special wings that help them go up in the air. They also have strong engines to push them forward.  
<end_of_turn>
<eos>


In [33]:
FastLanguageModel.for_inference(llama_model)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.add_eos_token = True

inputs = llama_tokenizer(
    [
        alpaca_prompt.format(
            f'''
            Your task is to answer children's questions using simple language.
            Explain any difficult words in a way a 3-year-old can understand.
            Keep responses under 60 words.
            \n\nQuestion: {question_4}
            ''',  # instruction
            "",  # input
            "",  # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

outputs = llama_model.generate(**inputs, max_new_tokens=256, temperature=0.6)
decoded_outputs = llama_tokenizer.batch_decode(outputs)

response_text = decoded_outputs[0]
# response_start = response_text.find("### Response:") + len("### Response:")
# initial_response = response_text[response_start:].strip()

match = re.search(r"### Response:(.*?)(?=\n###|$)", response_text, re.DOTALL)
if match:
    initial_response = match.group(1).strip()
else:
    initial_response = ""

print("Initial Response:", initial_response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Initial Response: Exercise is good for our bodies. When we exercise, we move our muscles. Moving our muscles helps them grow. It also helps us stay healthy. When we are healthy, we can do more things. We can run, jump, and climb. We can also eat more food and sleep well. Our bodies are made to move. If we don't move, our bodies won't grow. We will be sick and weak. We need to exercise to stay healthy. That's why we need to exercise.<|end_of_text|>


In [34]:
FastLanguageModel.for_inference(gemma_model)
gemma_tokenizer.pad_token = gemma_tokenizer.eos_token
gemma_tokenizer.add_eos_token = True

inputs = gemma_tokenizer(
    [
        alpaca_prompt.format(
            f'''
            Modify the given content for a 5-year-old.
            Use simple words and phrases.
            Remove any repetitive information.
            Keep responses under 50 words.
            \n\nGiven Content: {initial_response}
            ''',  # instruction
            "",  # input
            "",  # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

outputs = gemma_model.generate(**inputs, max_new_tokens=256, temperature=0.6)
decoded_outputs = gemma_tokenizer.batch_decode(outputs)

response_text = decoded_outputs[0]

match = re.search(r"### Response:(.*?)(?=\n###|$)", response_text, re.DOTALL)
if match:
    adjusted_response = match.group(1).strip()
else:
    adjusted_response = ""

print("Adjusted Response:", adjusted_response)

Adjusted Response: <eos> 

Exercise makes our bodies strong! When we move, our muscles grow big and help us do fun things like run and jump.  We need to exercise to be healthy and strong. 
<end_of_turn><eos>


In [36]:
FastLanguageModel.for_inference(llama_model)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.add_eos_token = True

inputs = llama_tokenizer(
    [
        alpaca_prompt.format(
            f'''
            Your task is to answer children's questions using simple language.
            Explain any difficult words in a way a 3-year-old can understand.
            Keep responses under 60 words.
            \n\nQuestion: {question_5}
            ''',  # instruction
            "",  # input
            "",  # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

outputs = llama_model.generate(**inputs, max_new_tokens=256, temperature=0.6)
decoded_outputs = llama_tokenizer.batch_decode(outputs)

response_text = decoded_outputs[0]
# response_start = response_text.find("### Response:") + len("### Response:")
# initial_response = response_text[response_start:].strip()

match = re.search(r"### Response:(.*?)(?=\n###|$)", response_text, re.DOTALL)
if match:
    initial_response = match.group(1).strip()
else:
    initial_response = ""

print("Initial Response:", initial_response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Initial Response: Computers are machines that process information. They use electricity to do this. Electricity is a form of energy that can be used to power devices. Computers are made up of many different components, such as a processor, memory, and storage. The processor is the brain of the computer, and it's responsible for carrying out instructions. Memory is used to store data and instructions, while storage is used to store data and programs. The computer receives information from a user through an input device, such as a keyboard or mouse. This information is processed by the processor and stored in memory. The computer then outputs the results to a display device, such as a monitor or printer. Computers can also communicate with other computers over a network or the internet. In this way, they can share information and resources.<|end_of_text|>


In [37]:
FastLanguageModel.for_inference(gemma_model)
gemma_tokenizer.pad_token = gemma_tokenizer.eos_token
gemma_tokenizer.add_eos_token = True

inputs = gemma_tokenizer(
    [
        alpaca_prompt.format(
            f'''
            Modify the given content for a 5-year-old.
            Use simple words and phrases.
            Remove any repetitive information.
            Keep responses under 50 words.
            \n\nGiven Content: {initial_response}
            ''',  # instruction
            "",  # input
            "",  # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

outputs = gemma_model.generate(**inputs, max_new_tokens=256, temperature=0.6)
decoded_outputs = gemma_tokenizer.batch_decode(outputs)

response_text = decoded_outputs[0]

match = re.search(r"### Response:(.*?)(?=\n###|$)", response_text, re.DOTALL)
if match:
    adjusted_response = match.group(1).strip()
else:
    adjusted_response = ""

print("Adjusted Response:", adjusted_response)

Adjusted Response: <eos>Computers are like magic boxes! They use electricity to think and do things. You can tell them what to do with a keyboard or mouse, and they show you the answer on a screen.  Computers can even talk to each other! 
<end_of_turn><eos>
